In [1]:
# Celda 1: Paquetes y Funciones Auxiliares
using LinearAlgebra
using RowEchelon # Necesario para la función rref()

"""
Calcula la pseudoinversa de Moore-Penrose para una matriz A utilizando el
algoritmo de factorización de rango completo.
"""
function pseudoinversa_full_rank(A::AbstractMatrix)
    # Convertimos a Float64 para compatibilidad con rref
    A_float = convert(Matrix{Float64}, A)
    
    # a) Reducir A a su forma escalonada reducida por filas (E_A)
    E_A = rref(A_float)
    
    # b) Formar la matriz B con las columnas pivote de A
    pivot_cols = []
    tol = 1e-8 
    for i in 1:size(E_A, 1)
        pivot_index = findfirst(x -> abs(x) > tol, E_A[i, :])
        if pivot_index !== nothing
            push!(pivot_cols, pivot_index)
        end
    end
    unique!(sort!(pivot_cols))
    B = A_float[:, pivot_cols]
    
    # c) Formar la matriz C con las filas no nulas de E_A
    nonzero_rows = [i for i in 1:size(E_A, 1) if any(x -> abs(x) > tol, E_A[i, :])]
    C = E_A[nonzero_rows, :]
    
    # d) y e) Calcular A† con la fórmula final
    B_star = B'
    C_star = C'
    
    # Evitar error si una matriz es invertible y la otra no
    try
        inv_B_star_B = inv(B_star * B)
        inv_C_C_star = inv(C * C_star)
        return C_star * inv_C_C_star * inv_B_star_B * B_star
    catch e
        # Si falla, es probable que A sea invertible, usamos pinv como respaldo
        return pinv(A_float)
    end
end

pseudoinversa_full_rank

In [2]:
# Celda 2: Definir la matriz y obtener eigenvalores
A1 = [2.0   -sqrt(3)/2;
      -sqrt(3)/2  1.0]

# Obtenemos los eigenvalores usando la función de Julia
D1_vals, P1_vecs_reales = eigen(A1)

println("--- Análisis del Ejercicio 1 ---")
println("Matriz A:")
display(A1)
println("\nEigenvalores encontrados: ", D1_vals)

--- Análisis del Ejercicio 1 ---
Matriz A:


2×2 Matrix{Float64}:
  2.0       -0.866025
 -0.866025   1.0


Eigenvalores encontrados: [0.5, 2.5]


In [3]:
# Celda 3: Calcular eigenvectores usando la pseudoinversa
println("\nCalculando eigenvectores con el método de la pseudoinversa:")

for lambda in D1_vals
    println("\nPara el eigenvalor λ = ", round(lambda, digits=4))
    
    # 1. Formar la matriz B = A - λI
    B = A1 - lambda * I
    
    # 2. Calcular la pseudoinversa de B
    B_pinv = pseudoinversa_full_rank(B)
    
    # 3. Calcular la matriz de proyección al espacio nulo: N = I - B†B
    N = I - B_pinv * B
    
    # 4. Extraer un eigenvector (una columna no nula de N)
    # Buscamos la columna con la norma más grande para evitar elegir una columna de ceros
    norms = [norm(N[:, i]) for i in 1:size(N, 2)]
    _, max_idx = findmax(norms)
    eigenvector = N[:, max_idx]
    
    # 5. Normalizar el vector
    eigenvector_normalizado = eigenvector / norm(eigenvector)
    
    println("  Eigenvector encontrado: ", round.(eigenvector_normalizado, digits=4))
end

println("\n--- Verificación ---")
println("Eigenvectores calculados por la función `eigen` de Julia:")
display(round.(P1_vecs_reales, digits=4))


Calculando eigenvectores con el método de la pseudoinversa:

Para el eigenvalor λ = 0.5
  Eigenvector encontrado: [0.5, 0.866]

Para el eigenvalor λ = 2.5
  Eigenvector encontrado: [0.866, -0.5]

--- Verificación ---
Eigenvectores calculados por la función `eigen` de Julia:


2×2 Matrix{Float64}:
 -0.5    -0.866
 -0.866   0.5

In [4]:
# Celda 4: Definir la matriz y obtener eigenvalores
A2 = [2.0 1.0 1.0;
      1.0 1.0 0.0;
      1.0 0.0 1.0]

D2_vals, P2_vecs_reales = eigen(A2)

println("\n\n--- Análisis del Ejercicio 2 ---")
println("Matriz A:")
display(A2)
println("\nEigenvalores encontrados: ", D2_vals)



--- Análisis del Ejercicio 2 ---
Matriz A:


3×3 Matrix{Float64}:
 2.0  1.0  1.0
 1.0  1.0  0.0
 1.0  0.0  1.0


Eigenvalores encontrados: [2.6645352591003757e-15, 1.0000000000000018, 3.0]


In [5]:
# Celda 5: Calcular eigenvectores usando la pseudoinversa
println("\nCalculando eigenvectores con el método de la pseudoinversa:")

for lambda in D2_vals
    println("\nPara el eigenvalor λ = ", round(lambda, digits=4))
    
    # 1. Formar la matriz B = A - λI
    B = A2 - lambda * I
    
    # 2. Calcular la pseudoinversa de B
    B_pinv = pseudoinversa_full_rank(B)
    
    # 3. Calcular la matriz de proyección al espacio nulo: N = I - B†B
    N = I - B_pinv * B
    
    # 4. Extraer un eigenvector (una columna no nula de N)
    norms = [norm(N[:, i]) for i in 1:size(N, 2)]
    _, max_idx = findmax(norms)
    eigenvector = N[:, max_idx]
    
    # 5. Normalizar el vector
    # Manejar el caso de un vector casi nulo por errores numéricos
    if norm(eigenvector) > 1e-8
        eigenvector_normalizado = eigenvector / norm(eigenvector)
        println("  Eigenvector encontrado: ", round.(eigenvector_normalizado, digits=4))
    else
        println("  No se pudo extraer un eigenvector claro (posiblemente por precisión numérica).")
    end
end

println("\n--- Verificación ---")
println("Eigenvectores calculados por la función `eigen` de Julia:")
display(round.(P2_vecs_reales, digits=4))


Calculando eigenvectores con el método de la pseudoinversa:

Para el eigenvalor λ = 0.0
  Eigenvector encontrado: [-0.6364, 0.5455, 0.5455]

Para el eigenvalor λ = 1.0
  Eigenvector encontrado: [-0.0, -0.7071, 0.7071]

Para el eigenvalor λ = 3.0
  Eigenvector encontrado: [0.8165, 0.4082, 0.4082]

--- Verificación ---
Eigenvectores calculados por la función `eigen` de Julia:


3×3 Matrix{Float64}:
 -0.5774  -0.0     -0.8165
  0.5774  -0.7071  -0.4082
  0.5774   0.7071  -0.4082

In [6]:
# Celda 6: Definir la matriz y obtener eigenvalores
A3 = [0.0 1.0 1.0;
      1.0 0.0 1.0;
      1.0 1.0 0.0]

# Usamos la función de Julia para obtener los valores de referencia
D3_vals, P3_vecs_reales = eigen(A3)

println("--- Análisis del Ejercicio 3 ---")
println("Matriz A:")
display(A3)
# Usamos `unique` para manejar el eigenvalor repetido en el bucle
println("\nEigenvalores encontrados: ", unique(D3_vals))

--- Análisis del Ejercicio 3 ---
Matriz A:


3×3 Matrix{Float64}:
 0.0  1.0  1.0
 1.0  0.0  1.0
 1.0  1.0  0.0


Eigenvalores encontrados: [-0.9999999999999999, -0.9999999999999982, 1.9999999999999998]


In [7]:
# Celda 7: Calcular eigenvectores para cada eigenvalor único
println("\nCalculando eigenvectores con el método de la pseudoinversa:")

for lambda in unique(D3_vals)
    println("\nPara el eigenvalor λ = ", round(lambda, digits=4))
    
    B = A3 - lambda * I
    B_pinv = pseudoinversa_full_rank(B)
    N = I - B_pinv * B
    
    println("  Matriz de Proyección al Espacio Nulo (N = I - B†B):")
    display(round.(N, digits=4))

    # Extraemos los vectores base del espacio nulo desde las columnas de N
    # Usamos `qr` para obtener una base ortonormal de las columnas de N,
    # que es un método robusto para extraer la base del subespacio.
    base_vectors = qr(N).Q[:, 1:rank(N)]
    
    println("  Base de eigenvectores encontrada:")
    for i in 1:size(base_vectors, 2)
        eigenvector = base_vectors[:, i]
        # La orientación puede ser opuesta (+/-), es igualmente válido
        println("    v$i = ", round.(eigenvector, digits=4))
    end
end

println("\n--- Verificación ---")
println("Eigenvectores calculados por la función `eigen` de Julia:")
display(round.(P3_vecs_reales, digits=4))


Calculando eigenvectores con el método de la pseudoinversa:

Para el eigenvalor λ = -1.0
  Matriz de Proyección al Espacio Nulo (N = I - B†B):


3×3 Matrix{Float64}:
  0.6667  -0.3333  -0.3333
 -0.3333   0.6667  -0.3333
 -0.3333  -0.3333   0.6667

  Base de eigenvectores encontrada:
    v1 = [-0.8165, 0.4082, 0.4082]
    v2 = [-0.0, -0.7071, 0.7071]

Para el eigenvalor λ = -1.0
  Matriz de Proyección al Espacio Nulo (N = I - B†B):


3×3 Matrix{Float64}:
  0.6667  -0.3333  -0.3333
 -0.3333   0.6667  -0.3333
 -0.3333  -0.3333   0.6667

  Base de eigenvectores encontrada:
    v1 = [-0.8165, 0.4082, 0.4082]
    v2 = [-0.0, -0.7071, 0.7071]

Para el eigenvalor λ = 2.0
  Matriz de Proyección al Espacio Nulo (N = I - B†B):


3×3 Matrix{Float64}:
 0.3333  0.3333  0.3333
 0.3333  0.3333  0.3333
 0.3333  0.3333  0.3333

  Base de eigenvectores encontrada:
    v1 = [-0.5774, -0.5774, -0.5774]

--- Verificación ---
Eigenvectores calculados por la función `eigen` de Julia:


3×3 Matrix{Float64}:
  0.7071  -0.4082  -0.5774
 -0.7071  -0.4082  -0.5774
  0.0      0.8165  -0.5774

In [8]:
# Celda 8: Definir la matriz y obtener eigenvalores
A4 = [3.0 0.0 2.0;
      0.0 -1.0 0.0;
      2.0 0.0 0.0]

D4_vals, P4_vecs_reales = eigen(A4)

println("\n\n--- Análisis del Ejercicio 4 ---")
println("Matriz A:")
display(A4)
println("\nEigenvalores encontrados: ", unique(D4_vals))



--- Análisis del Ejercicio 4 ---
Matriz A:


3×3 Matrix{Float64}:
 3.0   0.0  2.0
 0.0  -1.0  0.0
 2.0   0.0  0.0


Eigenvalores encontrados: [-1.0, -0.9999999999999964, 4.0]


In [9]:
# Celda 9: Calcular eigenvectores usando la pseudoinversa
println("\nCalculando eigenvectores con el método de la pseudoinversa:")

for lambda in unique(D4_vals)
    println("\nPara el eigenvalor λ = ", round(lambda, digits=4))
    
    B = A4 - lambda * I
    B_pinv = pseudoinversa_full_rank(B)
    N = I - B_pinv * B
    
    println("  Matriz de Proyección al Espacio Nulo (N = I - B†B):")
    display(round.(N, digits=4))

    # Extraemos la base ortonormal del espacio de columnas de N
    base_vectors = qr(N).Q[:, 1:rank(N)]

    println("  Base de eigenvectores encontrada:")
    for i in 1:size(base_vectors, 2)
        eigenvector = base_vectors[:, i]
        println("    v$i = ", round.(eigenvector, digits=4))
    end
end

println("\n--- Verificación ---")
println("Eigenvectores calculados por la función `eigen` de Julia:")
display(round.(P4_vecs_reales, digits=4))


Calculando eigenvectores con el método de la pseudoinversa:

Para el eigenvalor λ = -1.0
  Matriz de Proyección al Espacio Nulo (N = I - B†B):


3×3 Matrix{Float64}:
  0.2  -0.0  -0.4
 -0.0   1.0  -0.0
 -0.4  -0.0   0.8

  Base de eigenvectores encontrada:
    v1 = [-0.4472, 0.0, 0.8944]
    v2 = [0.0, 1.0, 0.0]

Para el eigenvalor λ = -1.0
  Matriz de Proyección al Espacio Nulo (N = I - B†B):


3×3 Matrix{Float64}:
 -0.0312  -0.0   0.0156
 -0.0      0.0  -0.0
  0.1875  -0.0   0.0312

  Base de eigenvectores encontrada:
    v1 = [-0.1644, 0.0, 0.9864]
    v2 = [0.0, 1.0, 0.0]

Para el eigenvalor λ = 4.0
  Matriz de Proyección al Espacio Nulo (N = I - B†B):


3×3 Matrix{Float64}:
  0.8  -0.0   0.4
 -0.0   0.0  -0.0
  0.4  -0.0   0.2

  Base de eigenvectores encontrada:
    v1 = [-0.8944, 0.0, -0.4472]

--- Verificación ---
Eigenvectores calculados por la función `eigen` de Julia:


3×3 Matrix{Float64}:
  0.0  -0.4472  -0.8944
 -1.0   0.0      0.0
  0.0   0.8944  -0.4472